In [4]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import linprog

```python
linprog(
    c: Any, 
    A_ub: Any | None = None, 
    b_ub: Any | None = None, 
    A_eq: Any | None = None, 
    b_eq: Any | None = None, 
    bounds: Any | None = None, 
    method: str = 'highs', 
    callback: Any | None = None, 
    options: Any | None = None, 
    x0: Any | None = None, 
    integrality: Any | None = None
) -> OptimizeResult
```

constraints: X >= 10 per week,

machine-time: 100
craftsman-time: 20
production capacity: 40 hours of machine time and 35 hours of craftsman

constraints:
$$c_1: \frac{15}{60}x + \frac{20}{60}y \leq 40$$
$$c_2: \frac{20}{60}x + \frac{30}{60}y \leq 35$$
$$c_3: x \geq 10$$
$$c_4: x, y \geq 0$$


revenues:
$$200 - (\frac{15}{60}*100 + \frac{20}{60}*20)$$
$$300 - (\frac{20}{60}*100 + \frac{30}{60}*20)$$






In [5]:
x = {
    "machine_time": 15,
    "craftsman_time": 20,
    "revenue": 200,
    "machine_cost": 15 / 60 * 100,
    "craftsman_cost": 20 / 60 * 20,
    "constraint_units": 10,
}

y = {
    "machine_time": 20,
    "craftsman_time": 30,
    "revenue": 300,
    "machine_cost": 20 / 60 * 100,
    "craftsman_cost": 30 / 60 * 20,
    "constraint_units": 0,
}

machine_time_constraint = 40
craftsman_time_constraint = 35

c = [
    -(x["revenue"] - (x["machine_cost"] + x["craftsman_cost"])),
    -(y["revenue"] - (y["machine_cost"] + y["craftsman_cost"])),
]

bounds = [(x["constraint_units"], None), (y["constraint_units"], None)]
A_ub = [
    [x["machine_time"] / 60, y["machine_time"] / 60],
    [x["craftsman_time"] / 60, y["craftsman_time"] / 60],
]
b_ub = [machine_time_constraint, craftsman_time_constraint]


linprog(c, bounds=bounds, A_ub=A_ub, b_ub=b_ub)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -17938.88888888889
              x: [ 1.000e+01  6.333e+01]
            nit: 1
          lower:  residual: [ 0.000e+00  6.333e+01]
                 marginals: [ 2.778e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 1.639e+01  0.000e+00]
                 marginals: [-0.000e+00 -5.133e+02]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [45]:
import re

vs = {
    "v1": "+xs1+x31-x14-x13",
    "v2": "+xs2-x25-x23",
    "v3": "+x13+x23+x43-x31-x35",
    "v4": "+x14+x54-x43-x4t",
    "v5": "+x25+x35-x54-x5t",
}

es = {
    "xs1": 14,
    "xs2": 25,
    "x13": 3,
    "x14": 21,
    "x23": 13,
    "x25": 7,
    "x31": 6,
    "x35": 15,
    "x43": 10,
    "x4t": 20,
    "x54": 5,
    "x5t": 10,

}

def get_edges(vs: dict):
    edges = {}
    
    for e in list(es.keys()):
        edges[e] = []
        for v in vs.values():
            search = re.search(e, v)
            if search:
                if v[search.start()-1] == "+":
                    edges[e].append(1)
                else:
                    edges[e].append(-1)
            else:
                edges[e].append(0)
    return edges

get_edges(vs)

c2 = [-1]*len(es)
bounds2 = [(0, e) for e in list(es.values())]
A_eq = np.array([v for v in get_edges(vs).values()]).T
b_eq = [0]*5

x = linprog(c2, bounds=bounds2, A_eq=A_eq, b_eq=b_eq)
x.x = [round(i) for i in x.x]

print(x)


        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -124.0
              x: [14, 16, 0, 20, 13, 3, 6, 12, 5, 20, 5, 10]
            nit: 4
          lower:  residual: [ 1.400e+01  1.600e+01  0.000e+00  2.000e+01
                              1.300e+01  3.000e+00  6.000e+00  1.200e+01
                              5.000e+00  2.000e+01  5.000e+00  1.000e+01]
                 marginals: [ 0.000e+00  0.000e+00  1.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00]
          upper:  residual: [ 0.000e+00  9.000e+00  3.000e+00  1.000e+00
                              0.000e+00  4.000e+00  0.000e+00  3.000e+00
                              5.000e+00  0.000e+00  0.000e+00  0.000e+00]
                 marginals: [-2.000e+00  0.000e+00  0.000e+00  0.000e+00
                             -1.000e+00  